In [411]:
import numpy as np
import math
import random
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [426]:
class Node:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y
        
    def distance(self, node):
        return math.sqrt((self.x-node.x)**2 + (self.y-node.y)**2)
    
class Individual:
    def __init__(self, path):
        self.path = path
        self.distance = 0.0
        self.fitness = 0.0
        
    def path_length(self):
        pl = 0
        for i in range(0, len(self.path)):
            node1 = self.path[i]
            node2 = None
            if i+1 < len(self.path):
                node2 = self.path[i+1]
            else:
                node2 = self.path[0]
            pl = pl + node1.distance(node2)
            self.distance = pl
            self.fitness = 1/float(pl)
        return pl
    
    def get_path(self):
        path_list = []
        for i in self.path:
            path_list.append(i.name)
        return path_list
    
def initialize_pop(pop_size, nodes):
    population = []
    for i in range(0, pop_size):
        route = random.sample(nodes, len(nodes))
        individual = Individual(route)
        population.append(individual)
    return population

def read_tsp(file_name):
    nodes = []
    with open(file_name, "r") as file:
        data = file.readlines()
        
    for i in data:
        split_i = i.split(" ")
        if len(split_i) == 3:
            nodes.append(Node(int(split_i[0]), float(split_i[1]), float(split_i[2])))
    return nodes

def find_fitnesses(population):
    for i in range(0, len(population)):
        population[i].path_length()

def rank_population_elite(population, gen_size):
    sorted_pop = sorted(population, key=lambda x: x.fitness, reverse=True)
    return sorted_pop[:gen_size]
    
def rank_population_proportional(population, gen_size):
    selected = []
    pop_list = []
    for i in range(0, len(population)):
        pop_list.append((i, population[i].fitness))
        
    df = pd.DataFrame(pop_list, columns=["Individual", "Fitness"])
    df['breeding_chance'] = df.Fitness/df.Fitness.sum()
    for i in range(0, gen_size):
        selection = random.uniform(0,1)
        j = 0
        while True:
            row = df.iloc[j]
            selection -= row['breeding_chance']
            if selection < 0:
                break
            j+=1
        selected_row = df.iloc[j]
        selected.append(int(selected_row['Individual']))
        selected_prob = selected_row['breeding_chance']
        df = df.drop(j)
        df = df.reset_index(drop=True)
        if len(df) > 0:
            df["breeding_chance"] += selected_prob/len(df)
        
    return [population[i] for i in selected]

def breed(parent1, parent2, nodes):
    
    p1_path = parent1.get_path()
    p2_path = parent2.get_path()
    
    gene1 = int(random.random()*len(p1_path))
    gene2 = int(random.random()*len(p1_path))
    
    start_idx = min(gene1, gene2)
    end_idx = max(gene1, gene2)
    
    segment_p1 = p1_path[start_idx:end_idx]
    segment_p2 = [i for i in p2_path if i not in segment_p1]
    
    new_path = segment_p1 + segment_p2
    child_path = []
    
    for i in new_path:
        for j in nodes:
            if j.name == i:
                child_path.append(Node(i, j.x, j.y))
    
    
    child = Individual(child_path)
    
    return Individual(child_path)

def breed_pop(pool, size, nodes):
    children = []
    
    for i in pool:
        children.append(i)
    
    while len(children) != size:
        parents = random.sample(pool, 2)
        child = breed(parents[0], parents[1], nodes)
        children.append(child)
        
    return children

def mutate(individual):
    num_nodes = len(individual.path)
    idx1 = int(random.random()*num_nodes)
    idx2 = int(random.random()*num_nodes) 
    individual.path[idx1], individual.path[idx2] = individual.path[idx2], individual.path[idx1]

def mutate_population(population, mut_rate):
    for i in population:
        chance = random.random()
        if chance <= mut_rate:
            mutate(i)

def make_generation(curr_gen, mut_rate, pop_size, select_size, selection_method, nodes):
    pop_to_breed = selection_method(curr_gen, select_size)
    new_generation = breed_pop(pop_to_breed, pop_size, nodes)
    mutate_population(new_generation, mut_rate)
    return new_generation

def run_ga_tsp(nodes, num_generations, mut_rate, pop_size, select_size, selection_method):
    start_time = time.time()
    avg_gen_fit_list = []
    best_gen_fit_list = []
    
    gen1 = initialize_pop(pop_size, nodes)
    find_fitnesses(gen1)
    
    best_fit = 0
    best_path = 0
    best_dist = math.inf
    
    for i in range(0, num_generations):
        new_gen = make_generation(pop, mut_rate, pop_size, select_size, selection_method, nodes)
        find_fitnesses(new_gen)
        
        best_gen_fit = 0
        sum_gen_fit = 0
        for j in new_gen:
            if j.fitness > best_gen_fit:
                best_gen_fit = j.fitness
            if best_gen_fit > best_fit:
                best_fit = best_gen_fit
                best_path = j.get_path()
            if j.distance < best_dist:
                best_dist = j.distance
            sum_gen_fit += j.fitness
        avg_gen_fit = sum_gen_fit/len(new_gen)
        best_gen_fit_list.append(best_gen_fit)
        avg_gen_fit_list.append(avg_gen_fit)
        
    print("Execution time: " + str(time.time()-start_time))
    
    fig, ax = plt.subplots(2, figsize=(12,12))
    ax[0].plot(range(1, len(avg_gen_fit_list)+1), avg_gen_fit_list)
    ax[0].set_title("Average Fitness by Generation")
    ax[1].plot(range(1, len(best_gen_fit_list)+1), best_gen_fit_list)
    ax[1].set_title("Best Fitness by Generation")
    
    fig.text(0.5, 0.04, "Generation", ha="center")
    fig.text(0.04, 0.5, "Fitness Scores", va="center", rotation=90)
    
    print("Best Overall Fitness: " + str(best_fit))
    print("Best Path: " + str(best_path))
    print("Best Overall Distance: " + str(best_dist))

In [ ]:
nodes = read_tsp("Random100.tsp")

run_ga_tsp(nodes, 1000, 0.01, 100, 10, rank_population_elite)